In [1]:
import sys
import pandas as pd
import configparser
from io import StringIO , BytesIO

sys.path.append('/home/renan/dev/python/project-tcc/dags/')
config = configparser.ConfigParser()
config.read('../notebooks/config-dev.ini')

['../notebooks/config-dev.ini']

In [2]:
from util import datalake
from util.data import creater_analysis, pre_processing

data_lake_user = config['azure']['data_lake_user']
data_lake_key = config['azure']['data_lake_key']
fs_name = config['datalake']['fs_name']

In [3]:
client = datalake.DataLake(data_lake_user, data_lake_key, fs_name)
client.list_files('raw_data')

['raw_data/2014',
 'raw_data/2014/Benefits_Cost_Sharing_PUF.csv',
 'raw_data/2014/Business_Rules_PUF.csv',
 'raw_data/2014/Network_PUF.csv',
 'raw_data/2014/Plan_Attributes_PUF_2014_2015-03-09.csv',
 'raw_data/2014/Rate_PUF.csv',
 'raw_data/2014/Service_Area_PUF.csv',
 'raw_data/2015',
 'raw_data/2015/Benefits_Cost_Sharing_PUF.csv',
 'raw_data/2015/Business_Rules_PUF_Reformat.csv',
 'raw_data/2015/Network_PUF.csv',
 'raw_data/2015/Plan_Attributes_PUF.csv',
 'raw_data/2015/Rate_PUF.csv',
 'raw_data/2015/Service_Area_PUF.csv',
 'raw_data/2016',
 'raw_data/2016/Benefits_Cost_Sharing_PUF_2015-12-08.csv',
 'raw_data/2016/Business_Rules_PUF_2015-12-08.csv',
 'raw_data/2016/Network_PUF_2015-12-08.csv',
 'raw_data/2016/Plan_Attributes_PUF_2015-12-08.csv',
 'raw_data/2016/Rate_PUF_2015-12-08.csv',
 'raw_data/2016/ServiceArea_PUF_2015-12-08.csv',
 'raw_data/data',
 'raw_data/data/Plan_Crosswalk_PUF_2014-12-22.csv',
 'raw_data/data/Plan_ID_Crosswalk_PUF_2015-12-07.CSV']

In [4]:
name_file = 'Benefits_Cost_Sharing_PUF'

In [5]:
buffer, _, metadata = client.get_file('raw_data/2014/', name_file + '.csv')
df = pd.read_csv(BytesIO(buffer))
md = metadata.metadata
md

{'content_hash': '2014_Benefits_Cost_Sharing_PUF.csv',
 'filename': 'Benefits_Cost_Sharing_PUF.csv',
 'id': '3',
 'schema': 'BenefitsCostSharing',
 'year': '2014'}

In [6]:
new_name_file = md['schema'] + "_validator.json"

df_validator = pd.read_json("../data/json/" + new_name_file) 
df_validator = df_validator.rename(columns=lambda x: x.split('_', 1)[1])
display(df_validator.head(5))

,column_name,is_anonymizated_field,is_primary_key,column_name_formatted,percent_filling,column_type_dataframe,column_type,column_size,column_percent_unique
0,BusinessYear,False,False,BUSINESSYEAR,100.0,int64,INTEGER,NaN,0.01
1,StateCode,False,False,STATECODE,100.0,object,VARCHAR,2.0,0.02
2,IssuerId,False,False,ISSUERID,100.0,int64,INTEGER,NaN,0.13
3,SourceName,False,False,SOURCENAME,100.0,object,VARCHAR,4.0,0.01
4,VersionNum,False,False,VERSIONNUM,100.0,int64,INTEGER,NaN,0.08


In [7]:
from util.data import schema_validator, creater_analysis


In [8]:
df_analysis = creater_analysis.create_schema_analysis(df)
display(df_analysis.head(5))

,column_name,column_name_formatted,percent_filling,column_type_dataframe,column_type,column_size,column_percent_unique
0,BusinessYear,BUSINESSYEAR,100.0,int64,INTEGER,None,0.05
1,StateCode,STATECODE,100.0,object,VARCHAR,2,0.05
2,IssuerId,ISSUERID,100.0,int64,INTEGER,None,0.25
3,SourceName,SOURCENAME,100.0,object,VARCHAR,4,0.05
4,VersionNum,VERSIONNUM,100.0,int64,INTEGER,None,0.20


In [9]:
df_validator = df_validator.drop(df_validator.index[0])
df_validator.head(2)

,column_name,is_anonymizated_field,is_primary_key,column_name_formatted,percent_filling,column_type_dataframe,column_type,column_size,column_percent_unique
1,StateCode,False,False,STATECODE,100.0,object,VARCHAR,2.0,0.02
2,IssuerId,False,False,ISSUERID,100.0,int64,INTEGER,NaN,0.13


In [10]:
validationData = schema_validator.SchemaValidator(df_analysis, df_validator)

In [11]:
df_validator.loc[:4, 'is_anonymizated_field'] = True

In [12]:
filtered_values = df_validator.loc[df_validator['is_anonymizated_field'], 'column_name'].values
filtered_values

array(['StateCode', 'IssuerId', 'SourceName', 'VersionNum'], dtype=object)

In [11]:
# Step 1 - Validate the database
display(validationData.get_analysis_df().head(4))

,column_name,column_name_formatted,percent_filling,column_type_dataframe,column_type,column_size,column_percent_unique
0,BusinessYear,BUSINESSYEAR,100.0,int64,INTEGER,None,0.05
1,StateCode,STATECODE,100.0,object,VARCHAR,2,0.05
2,IssuerId,ISSUERID,100.0,int64,INTEGER,None,0.25
3,SourceName,SOURCENAME,100.0,object,VARCHAR,4,0.05


In [12]:
validationData.get_validator_df().shape[0]

31

In [13]:
validationData.get_analysis_filted_df().shape[0]

31

In [15]:
print(validationData.check_if_dataframe_is_empty())

None


In [15]:
# Step 4 - Validate data to check missing values
# (None, no have missing - True have missing)
print(validationData.check_for_missing_values())


O arquivo tem colunas importantes vazias: ['CopayInnTier2', 'CoinsInnTier2', 'MinimumStay']


In [16]:
print(validationData.validate_number_fields())

None


In [17]:
# Step 2 - Processing the matched columns
# Validate the matched columns (True, no match - False, match )
print(validationData.validate_matched_columns('column_name'))

None


In [19]:
# Step 3 - Validate the data types
# Validate datatype (True, match type ok, False, match type
print(validationData.validate_data_types("column_name", 'column_type_dataframe'))

Os tipos das colunas do arquivo estão como o esperado.


In [20]:
result_check_unique_primary_key = validationData.check_unique_primary_key()
print(result_check_unique_primary_key)

None
